In [1]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.externals import joblib
import math

In [2]:
products = pd.read_csv('amazon_baby_subset.csv')

In [5]:
products.head(3)

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1


In [4]:
products['name'].head(10)

0    Stop Pacifier Sucking without tears with Thumb...
1      Nature's Lullabies Second Year Sticker Calendar
2      Nature's Lullabies Second Year Sticker Calendar
3                          Lamaze Peekaboo, I Love You
4    SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                            Our Baby Girl Memory Book
6    Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7    Blessed By Pope Benedict XVI Divine Mercy Full...
8    Cloth Diaper Pins Stainless Steel Traditional ...
9    Cloth Diaper Pins Stainless Steel Traditional ...
Name: name, dtype: object

In [15]:
print(products.sentiment.value_counts())

 1    26579
-1    26493
Name: sentiment, dtype: int64


## Cleaning

In [16]:
def remove_punctuation(text):
    translator = text.maketrans('','',string.punctuation)
    return text.translate(translator)

In [17]:
products.review.fillna('',inplace=True)

In [18]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [19]:
products.iloc[0]['review_clean']

'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

In [41]:
important_words = pd.read_json('important_words.json')[0]
important_words = list(important_words)

In [42]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [46]:
products['contains_perfect'] = np.where(products['perfect']>=1,1,0)

In [48]:
print(products.contains_perfect.sum(),'reviews contain the word perfect')

2955 reviews contain the word perfect


## Convert DataFrame to multi-dimensional array 

In [49]:
def get_numpy_data(df, features, label):
    df['constant'] = 1
    features = ['constant'] + features
    features_frame = df[features]
    feature_matrix = features_frame.as_matrix()
    label_sarray = df[label]
    label_array = label_sarray.as_matrix()
    return(feature_matrix, label_array)

In [51]:
feature_matrix, sentiment = get_numpy_data(products,important_words,'sentiment')

In [58]:
print('There are',feature_matrix.shape[1],'features in feature_matrix')

There are 194 features in feature_matrix


## Estimating conditional probability with link function

In [147]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''

def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    score = np.dot(feature_matrix, coefficients)

    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    predictions = 1/(1+np.exp(score))
    
    # return predictions
    return score

## Compute derivative of log likelihood with respect to a single coefficient

In [60]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors,feature)
        # Return the derivative
    return derivative

In [62]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

## Taking Gradient Steps

In [167]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = np.array(initial_coefficients) # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, initial_coefficients)

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors,feature_matrix[:,j])
            print(derivative)

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
#             coefficients[j] = step_si÷ze*derivative

        # Checking whether log likelihood is increasing
#         if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
#         or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
#             lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
#             print('iteration %*d: log likelihood of observed labels = %.8f' % \
#                 (int(np.ceil(np.log10(max_iter))), itr, lp))
    return errors

In [170]:
len(logistic_regression(feature_matrix, sentiment, np.zeros(194), 1e-7, 1))

26579.0
9822.0
9807.0
8113.0
5997.0
7503.0
6991.0
6812.0
6165.0
6438.0
6084.0
5632.0
5370.0
5356.0
4123.0
4612.0
4371.0
4318.0
4055.0
4199.0
3717.0
3823.0
3940.0
3619.0
3713.0
3925.0
3673.0
3398.0
3355.0
3201.0
3366.0
3239.0
2926.0
2860.0
2612.0
2666.0
2855.0
2593.0
2700.0
2701.0
2534.0
2725.0
2120.0
2546.0
2202.0
2219.0
2187.0
2371.0
2229.0
2128.0
2119.0
2238.0
1994.0
2271.0
2074.0
2328.0
2239.0
2178.0
2190.0
2132.0
2017.0
2052.0
2159.0
2011.0
2095.0
1998.0
1992.0
1996.0
1873.0
1844.0
1930.0
1887.0
1746.0
1960.0
1733.0
1759.0
1650.0
1952.0
1884.0
1734.0
1725.0
1768.0
1759.0
1821.0
1654.0
1676.0
1776.0
1579.0
1736.0
1498.0
1651.0
1572.0
1639.0
1497.0
1421.0
1440.0
1513.0
882.0
1290.0
1227.0
1040.0
1357.0
1072.0
1334.0
1180.0
453.0
197.0
1162.0
1298.0
1344.0
1361.0
1657.0
1346.0
134.0
134.0
165.0
1449.0
1274.0
1545.0
1248.0
1442.0
1241.0
899.0
745.0
1469.0
1089.0
1566.0
1489.0
1518.0
1171.0
896.0
907.0
876.0
1103.0
379.0
777.0
1305.0
923.0
1092.0
737.0
469.0
937.0
1341.0
1435.0
1460.0
1

53072

In [172]:
feature_matrix.shape

(53072, 194)

In [68]:
range(10)

range(0, 10)